In [1]:
from paretoKnapsackTeams import *
import dill, sys

sys.path.append('../../datasets/pickled_data/')
data_path = '../../datasets/pickled_data/influence/'
    
#Import pickled data
with open(data_path + 'influence_data.dill', "rb") as fp:
    influence_data = dill.load(fp)

In [ ]:
influence_data.num_items

### Average Plotting across Samples